<a href="https://colab.research.google.com/github/ichekhovskikh/doc2vec/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Выполнить ипморт пакетов

!pip install pymorphy2[fast]

import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem
from IPython.display import HTML, display
from tabulate import tabulate

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

In [0]:
#@title Определить функцию лемматизации

morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

In [0]:
#@title Определить функцию удаление стоп слов

russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

In [0]:
#@title Определить функцию предобработки текста

def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = list(lemmatize(normalized_text))
    normalized_text = remove_stopwords(normalized_text)
    return normalized_text

In [0]:
#@title Определить функцию загрузки корпуса текстовых документов

def read_corpus(corpus_path, tokens_only=False):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            normalized_text = advanced_preprocess(article['text'])
            if tokens_only:
                yield normalized_text
            else:
                yield gensim.models.doc2vec.TaggedDocument(normalized_text, article['tags'])

In [0]:
#@title Определить функцию получения исходного текста статьи по индексу

def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

In [0]:
#@title Определить функцию получения исходного текста статьи по идентификатору

def get_article_text_by_id(id, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for index in range(len(corpus)):
          if (corpus[index]['id'] == id):
            return corpus[index]['text']
        return ""

In [0]:
#@title Определить функцию получения категории статьи по индексу

def get_article_class_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['class_name']

In [0]:
#@title Определить функцию получения категории статьи по идентификатору

def get_article_class_by_id(id, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for index in range(len(corpus)):
          if (corpus[index]['id'] == id):
            return corpus[index]['class_name']
        return ""

#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 500 научных статей на 5 различных тем.

Для тестирования будет использоваться тестовый корпус из 50 статей (10 статей на каждую тему).

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать), class_name (класс текста, был размечен вручную, необходим только для тестирования)

In [0]:
#@title Введите путь к файлам исходной базы статей:

train_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/train_corpus.json' #@param {type: "string"}
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/test_corpus.json' #@param {type: "string"}

train_corpus = []
test_corpus = []

try:
    train_corpus = list(read_corpus(train_path))
    test_corpus = list(read_corpus(test_path, tokens_only=True))
    print("Загружено.")
except:
    print("Невозможно открыть файл! Проверьте формат выбранных файлов.")

In [0]:
#@title Давайте посмотрим на учебный корпус:

print(train_corpus[:2])

In [0]:
#@title Корпус тестирования выглядит так:

print(test_corpus[:2])

Обратите внимание, что корпус тестирования представляет собой просто список списков и не содержит никаких тегов.

# Обучение модели
## Создание объекта Doc2Vec
Теперь мы создадим модель Doc2Vec с векторным размером 80 слов и перебираем учебный корпус 200 раз (данные параметры были получены после проведения ряда исследований).

Словарь содержит в себе все уникальные слова, извлеченных из учебного корпуса.

In [0]:
#@title Укажите параметры обучения модели:

vector_size = 80 #@param
window =  3 #@param
epochs =  200 #@param
alpha = 0.001 #@param
learning_method = 'PV-DM' #@param ["PV-DM", "PV-DBOW"] {type:"string"}

dm = 1
if (learning_method == 'PV-DBOW'):
    dm = 0

model = gensim.models.doc2vec.Doc2Vec(
    vector_size=vector_size, 
    dm=dm,
    window=window,
    alpha=alpha, 
    epochs=epochs)

model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

## Inferring a Vector
Важно отметить, что теперь вы можете вывести вектор для любого фрагмента текста без необходимости переобучать модель, передав список слов в функцию model.infer_vector. Затем этот вектор можно сравнить с другими векторами по косинусной близости.

In [0]:
#@title Получить вектор текста:

text_sample = "Горностай небольшой зверек семейства куньих очень ценится" #@param {type:"string"}

model.infer_vector(advanced_preprocess(text_sample))

Обратите внимание, что, поскольку лежащие в основе алгоритмы обучения представляют собой подходы на основе итеративной аппроксимации, в которой используется внутренняя рандомизация, таким обрахрм, повторные выводы одного и того же текста будут возвращать слегка разные векторы.

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [0]:
#@title Проверить модель на адекватьность на глаз

doc_id = random.randint(0, len(train_corpus) - 1)

inferred_vector = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([inferred_vector], topn=5)
tabledata = [["Индекс", "Схожесть", "Категория", "Текст"]]

print('\nТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}) «{}»: «{}»\n'.format(doc_id, get_article_class_by_id(doc_id, train_path), get_article_text_by_index(doc_id, train_path)))
for label, index in [('1)', 0), ('2)', 1), ('3)', 2), ('4)', 3), ('5)', 4)]:
    tabledata.append([label, sims[index], get_article_class_by_id(int(sims[index][0]), train_path), get_article_text_by_id(int(sims[index][0]), train_path)])

print(tabulate(tabledata))

Обратите внимание, что наиболее похожий документ (как правило, тот же текст) имеет параметр сходства, приближающийся к единице. 

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью с помощью f1-меры.

In [0]:
#@title Выполнить тестирование обученной модели

from sklearn.metrics import f1_score

test_size = len(test_corpus)

predicted_classes = [] 
test_classes = []

for doc_index in range(test_size):
    inferred_vector = model.infer_vector(test_corpus[doc_index])
    sims = model.docvecs.most_similar([inferred_vector], topn=5)

    test_article_class = get_article_class_by_index(doc_index, test_path)
    tabledata = [["Индекс", "Схожесть", "Категория", "Текст"]]
    print('\nТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}) «{}»: «{}»\n'.format(doc_index, test_article_class, get_article_text_by_index(doc_index, test_path)))
    for label, index in [('1)', 0), ('2)', 1), ('3)', 2), ('4)', 3), ('5)', 4)]:
        predicted_article_class = get_article_class_by_id(int(sims[index][0]), train_path)
        tabledata.append([label, sims[index], predicted_article_class, get_article_text_by_id(int(sims[index][0]), train_path)])
        predicted_classes.append(predicted_article_class)
        test_classes.append(test_article_class)
    print(tabulate(tabledata))

print('f1score: {}'.format(f1_score(test_classes, predicted_classes, average='macro')))

# Поиск похожих научных документов
Выполните поиск похожих научных статей

In [0]:
#@title Укажите путь к тексту статьи в формате *.txt или введите текст статьи:
article_text = '' #@param {type: "string"}
article_path = '' #@param {type: "string"}

try:
    if (article_text == ''):
        with urllib.request.urlopen(article_path) as article_url:
            article_text = article_url.read().decode()

    normalized_text = advanced_preprocess(article_text)
    inferred_vector = model.infer_vector(normalized_text)
    sims = model.docvecs.most_similar([inferred_vector], topn=5)

    tabledata = [["Индекс", "Схожесть", "Текст"]]
    print('\nТЕКСТ ИСХДНОГО ДОКУМЕНТА: «{}»\n'.format(article_text))
    for label, index in [('1)', 0), ('2)', 1), ('3)', 2), ('4)', 3), ('5)', 4)]:
        tabledata.append([label, sims[index], get_article_text_by_id(int(sims[index][0]), train_path)])
    print(tabulate(tabledata))
except:
  print("Ошибка поиска! Проверьте правильность ввода.")